In [13]:
from pyspark.sql import SparkSession

In [14]:
spark = SparkSession .\
builder .\
appName("spark") .\
config("spark.sql.warehouse.dir","/user/itv016843/warehouse") .\
enableHiveSupport() .\
master("yarn") .\
getOrCreate()

In [15]:
spark

In [16]:
! hadoop fs -ls /public/trendytech/lendingclubproject/raw

Found 4 items
drwxr-xr-x   - itv005857 supergroup          0 2023-09-15 14:40 /public/trendytech/lendingclubproject/raw/customers_data_csv
drwxr-xr-x   - itv005857 supergroup          0 2023-09-17 22:57 /public/trendytech/lendingclubproject/raw/loans_data_csv
drwxr-xr-x   - itv005857 supergroup          0 2023-09-18 07:32 /public/trendytech/lendingclubproject/raw/loans_defaulters_csv
drwxr-xr-x   - itv005857 supergroup          0 2023-09-18 07:31 /public/trendytech/lendingclubproject/raw/loans_repayments_csv


In [17]:
from pyspark.sql.functions import *

In [18]:
from pyspark.sql import *

In [19]:
df1 = spark.read \
.format("csv") \
.option("header",True) \
.load("/public/trendytech/lendingclubproject/raw/loans_repayments_csv")

In [20]:
df1.limit(5)

loan_id,total_rec_prncp,total_rec_int,total_rec_late_fee,total_pymnt,last_pymnt_amnt,last_pymnt_d,next_pymnt_d
141581221,1055.81,2591.7,0.0,3647.51,709.23,Mar-2019,Apr-2019
141506948,1252.75,306.04,0.0,1558.79,312.63,Mar-2019,Apr-2019
141357400,626.37,354.96,0.0,981.33,197.27,Mar-2019,Apr-2019
139445427,1118.16,297.36,0.0,1415.52,283.95,Mar-2019,Apr-2019
141407409,1169.72,3605.3,0.0,4775.02,964.9,Mar-2019,Apr-2019


In [21]:
df2 = df1.withColumn("Time",current_timestamp())

In [22]:
df2.limit(5)

loan_id,total_rec_prncp,total_rec_int,total_rec_late_fee,total_pymnt,last_pymnt_amnt,last_pymnt_d,next_pymnt_d,Time
28674390,7884.28,5178.41,0.0,14329.62,482.23,Nov-2016,null,2025-07-12 08:49:...
28563513,9000.0,955.61,0.0,9955.61,6420.29,Nov-2015,null,2025-07-12 08:49:...
28694706,22125.0,751.39,0.0,22876.39,22073.85,Dec-2014,null,2025-07-12 08:49:...
28722876,3196.23,1259.77,0.0,4639.79,178.24,Nov-2016,null,2025-07-12 08:49:...
28644402,26400.0,2150.7,0.0,28550.7,19292.99,Oct-2015,null,2025-07-12 08:49:...


In [23]:
df3 = df2.filter("total_pymnt is null").count()

In [24]:
df3

46

In [25]:
columns = [
    "loan_id",
    "total_rec_prncp",
    "total_rec_int",
    "total_rec_late_fee",
    "total_pymnt",
    "last_pymnt_amnt",
    "last_pymnt_d",
    "next_pymnt_d"
]

In [26]:
df4 = df2.na.drop(subset= columns)

In [27]:
df4.filter("total_pymnt is null").count()

0

In [28]:
df4.limit(5)

loan_id,total_rec_prncp,total_rec_int,total_rec_late_fee,total_pymnt,last_pymnt_amnt,last_pymnt_d,next_pymnt_d,Time
28654395,18134.86,10868.86,0.0,29003.72,547.24,Mar-2019,Apr-2019,2025-07-12 08:49:...
28714308,10134.68,4648.3,0.0,14782.98,279.1,Mar-2019,Apr-2019,2025-07-12 08:49:...
7383129,14472.21,5487.4,0.0,19959.61,375.33,Mar-2019,Apr-2019,2025-07-12 08:49:...
28613524,12062.42,6639.6,0.0,18702.02,352.41,Mar-2019,Apr-2019,2025-07-12 08:49:...
27512265,10157.19,4456.22,0.0,14613.41,275.2,Mar-2019,Apr-2019,2025-07-12 08:49:...


In [29]:
df4.filter("total_rec_int !=0.0 and total_pymnt == 0.0").count()

112

In [30]:
df4.limit(5)

loan_id,total_rec_prncp,total_rec_int,total_rec_late_fee,total_pymnt,last_pymnt_amnt,last_pymnt_d,next_pymnt_d,Time
28654395,18134.86,10868.86,0.0,29003.72,547.24,Mar-2019,Apr-2019,2025-07-12 08:50:...
28714308,10134.68,4648.3,0.0,14782.98,279.1,Mar-2019,Apr-2019,2025-07-12 08:50:...
7383129,14472.21,5487.4,0.0,19959.61,375.33,Mar-2019,Apr-2019,2025-07-12 08:50:...
28613524,12062.42,6639.6,0.0,18702.02,352.41,Mar-2019,Apr-2019,2025-07-12 08:50:...
27512265,10157.19,4456.22,0.0,14613.41,275.2,Mar-2019,Apr-2019,2025-07-12 08:50:...


In [31]:
from pyspark.sql.functions import *

In [32]:
from pyspark.sql import *

In [33]:
df5 = df4.withColumn(
    "total_pymnt",
    when(
        (col("total_rec_prncp") != 0.0) & (col("total_pymnt") == 0.0),
        col("total_rec_prncp") + col("total_rec_int") + col("total_rec_late_fee")
    ).otherwise(col("total_pymnt"))
)

In [34]:
df5.filter("total_rec_prncp ==0.0").limit(5)

loan_id,total_rec_prncp,total_rec_int,total_rec_late_fee,total_pymnt,last_pymnt_amnt,last_pymnt_d,next_pymnt_d,Time
140764094,0.0,525.03,27.29,552.32,572.99,Dec-2018,Apr-2019,2025-07-12 08:50:...
1067266,0.0,0.0,12290.6400143918,13.0,0.0,3890.64,0.0,2025-07-12 08:50:...
746036,0.0,0.0,16794.8541507005,27.0,0.0,1794.85,0.0,2025-07-12 08:50:...
740743,0.0,32576.740063679103,32111.36,f,0.0,0.0,0.0,2025-07-12 08:50:...
533035,0.0,0.0,28336.187051024102,24.0,0.0,4336.19,0.0,2025-07-12 08:50:...


In [35]:
df5.printSchema()

root
 |-- loan_id: string (nullable = true)
 |-- total_rec_prncp: string (nullable = true)
 |-- total_rec_int: string (nullable = true)
 |-- total_rec_late_fee: string (nullable = true)
 |-- total_pymnt: string (nullable = true)
 |-- last_pymnt_amnt: string (nullable = true)
 |-- last_pymnt_d: string (nullable = true)
 |-- next_pymnt_d: string (nullable = true)
 |-- Time: timestamp (nullable = false)



In [36]:
df6 = df5.filter("total_rec_prncp != '0.0' and total_pymnt != 'f'")

In [39]:
df6.limit(5)

loan_id,total_rec_prncp,total_rec_int,total_rec_late_fee,total_pymnt,last_pymnt_amnt,last_pymnt_d,next_pymnt_d,Time
141581221,1055.81,2591.7,0.0,3647.51,709.23,Mar-2019,Apr-2019,2025-07-12 08:50:...
141506948,1252.75,306.04,0.0,1558.79,312.63,Mar-2019,Apr-2019,2025-07-12 08:50:...
141357400,626.37,354.96,0.0,981.33,197.27,Mar-2019,Apr-2019,2025-07-12 08:50:...
139445427,1118.16,297.36,0.0,1415.52,283.95,Mar-2019,Apr-2019,2025-07-12 08:50:...
141407409,1169.72,3605.3,0.0,4775.02,964.9,Mar-2019,Apr-2019,2025-07-12 08:50:...


In [42]:
df7 = df6.withColumn("last_pymnt_d",
                    when(col("last_pymnt_d") == 0.0,"None").otherwise(col("last_pymnt_d"))
                     
                     
                    )

In [49]:
df8 = df7.withColumn("next_pymnt_d",
                    when(col("next_pymnt_d") == 0.0,"None").otherwise(col("next_pymnt_d"))
                     
                     
                    )

In [50]:
df8.filter("next_pymnt_d == 0.0")

loan_id,total_rec_prncp,total_rec_int,total_rec_late_fee,total_pymnt,last_pymnt_amnt,last_pymnt_d,next_pymnt_d,Time
